##### This is just a sample code so that you can understand how to use function calling it not a starter code 

In [1]:
import openai
import os
import json 
import requests
from pprint import pprint
import tiktoken
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-4-1106-preview"

##### In the example below LLM acts as an intermediary that translates a user's natural language request into a structured SQL query, which is then executed by a database tool to fetch and return the relevant data.


In [2]:
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
_ = load_dotenv(find_dotenv())
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [3]:
# import gcp 
_SCOPE = 'https://www.googleapis.com/auth/cloud-platform'
from google.cloud import bigquery
client = bigquery.Client.from_service_account_json('../../data_warehousing/data_warehousing/include/gcp/service_account.json')

In [4]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if tools is not None:
        json_data.update({"tools": tools})
    if tool_choice is not None:
        json_data.update({"tool_choice": tool_choice})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [5]:
from openai import OpenAI
def get_gpt_response(system_message, user_message):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {
                "role": "system",
                "content": system_message
            },
            {
                "role": "user",
                "content": user_message
            }
        ],
        temperature=0,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message.content

In [6]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "tool": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "tool":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


In [7]:
def get_table_names(client, dataset_id):
    """Return a list of table names in the specified dataset."""
    table_names = []
    dataset_ref = client.dataset(dataset_id)
    # List tables in the dataset
    tables = client.list_tables(dataset_ref)
    for table in tables:
        table_names.append(table.table_id)
    return table_names


def get_column_names(client, dataset_id, table_name):
    """Return a list of column names for the specified table."""
    column_names = []
    table_ref = client.dataset(dataset_id).table(table_name)
    table = client.get_table(table_ref)
    for field in table.schema:
        column_names.append(field.name)
    return column_names

def get_database_info(client, dataset_id):
    """Return a list of dicts containing the table name and columns for each table in the dataset."""
    table_dicts = []
    for table_name in get_table_names(client, dataset_id):
        column_names = get_column_names(client, dataset_id, table_name)
        table_dicts.append({"table_name": table_name, "column_names": column_names})
    return table_dicts


In [8]:
database_schema_dict = get_database_info(client,'youtube')
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

In [9]:
print(database_schema_string)

Table: dim_dail_view_7day
Columns: Date, RollingAverageViews
Table: dim_top10
Columns: Date, TotalViews
Table: raw_cities
Columns: Cities, City_name, Geography, Geography_3, Views, Watch_time__hours_, Average_view_duration
Table: raw_gender
Columns: Date, Views, Watch_time__hours_, Average_view_duration
Table: raw_total
Columns: Date, Views


In [20]:
prompts = """
# General Analysis Queries
1. "What are the most significant trends in viewership data over the past month? Consider views, watch time, and average view duration across different tables."
2. "Identify the top-performing videos in the last quarter based on data from the dim_top10 table."
...

# Specific Data Requests
3. "Provide a breakdown of views, watch time, and average view duration by device type for the past week, using data from raw_total and raw_gender tables."
4. "Show me the top 5 traffic sources driving the most views to the channel based on available data."
...

# SQL Query Generation
5. "Generate an SQL query to retrieve specific data points from specified tables. Clearly state the desired data points and conditions."
6. "Translate this natural language question into an SQL query: [Ask your question in plain English, specifying relevant tables]."
...

# Data Clarification
7. "Could you please clarify what you mean by X, and which tables or columns are relevant to your query?"
8. "Provide an example of what you're looking for, specifying the tables and columns you'd like to focus on."
...

# Uncertainty Handling
9. "I'm not confident in providing an accurate answer to that question based on available data. Would you like me to generate an approximate response or seek further input?"
10. "I'm unable to generate a query for that question due to ambiguity or potential data limitations. Could you please rephrase it or provide more specific details, including relevant tables and columns?"
...

# Additional Tables and Descriptions
- youtube.dim_dail_view_7day: Daily Views Trend (7-Day Rolling Average)
  Columns: Date, RollingAverageViews
  Example Table Data: 2020-09-02 18.857142857142858

- youtube.dim_top10: Top 10 Days by Views
  Columns: Date, TotalViews
  Example Table Data: 2022-08-09 311

- youtube.raw_cities: List of all cities
  Columns: Cities, City_name, Geography, Geography_3, Views, Watch_time__hours_, Average_view_duration
 Example Table Data:
    - 0x164b85cef5ab402d:0x8467b6b Addis Ababa, ET, ET-AA, 1252, 127.5042, 00:06:06

- youtube.raw_total: Total views by daily Date column is in this format yyy-mm-dd
  Columns: Date, Views
  Example Table Data: 2020-07-10 0

"""

In [21]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "ask_database",
            "description": "Use this function to answer user questions about youtube. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    }
]

In [22]:
system_message = f"""
<prompts>
{prompts}
</prompts>
"""

In [23]:
def count_token(input, model_name=GPT_MODEL, debug=False):
        encoding = tiktoken.encoding_for_model(model_name)
        codex = " ".join(input.splitlines())  # Join lines into a single string
        num_tokens = len(encoding.encode(codex))
        return num_tokens

In [24]:
count_token(system_message)

549

In [25]:
def ask_database(client, query):
    """Function to query BigQuery dataset with a provided SQL query."""
    try:
        query_job = client.query(query)
        results = query_job.result()
        rows = [row.values() for row in results]
    except Exception as e:
        results = f"query failed with error: {e}"
        rows = []
    return rows

def execute_function_call(message):
    if message["tool_calls"][0]["function"]["name"] == "ask_database":
        query = json.loads(message["tool_calls"][0]["function"]["arguments"])["query"]
        results = ask_database(client, query)
    else:
        results = f"Error: function {message['tool_calls'][0]['function']['name']} does not exist"
    return results

In [26]:
user_message = "Hi, What are the top 5 cities?"

# Messages list
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]

# Get GPT response
gpt_response = get_gpt_response(system_message, user_message)

# Add GPT response to messages
messages.append({"role": "assistant", "content": gpt_response})

# Assuming chat_completion_request returns a response with tool calls
chat_response = chat_completion_request(messages, tools)

# Extract assistant's message
assistant_message = chat_response.json()["choices"][0]["message"]

# Uncomment the following lines to print the tool call content
# print("=================== Tool Call Content ===================")
# print(assistant_message["tool_calls"][0]["function"])

# Execute the tool function call
if assistant_message.get("tool_calls"):
    results = execute_function_call(assistant_message)
    messages.append({
        "role": "tool",
        "tool_call_id": assistant_message["tool_calls"][0]['id'],
        "name": assistant_message["tool_calls"][0]["function"]["name"],
        "content": results
    })

# Print the conversation
print("=================== Conversation ===================")
pretty_print_conversation(messages)


=================== Conversation ===================
system: 
<prompts>

# General Analysis Queries
1. "What are the most significant trends in viewership data over the past month? Consider views, watch time, and average view duration across different tables."
2. "Identify the top-performing videos in the last quarter based on data from the dim_top10 table."
...

# Specific Data Requests
3. "Provide a breakdown of views, watch time, and average view duration by device type for the past week, using data from raw_total and raw_gender tables."
4. "Show me the top 5 traffic sources driving the most views to the channel based on available data."
...

# SQL Query Generation
5. "Generate an SQL query to retrieve specific data points from specified tables. Clearly state the desired data points and conditions."
6. "Translate this natural language question into an SQL query: [Ask your question in plain English, specifying relevant tables]."
...

# Data Clarification
7. "Could you please clarify 

In [27]:
user_message = "What is the name of the citie with Geography that equal to ET?"

# Messages list
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]

# Get GPT response
gpt_response = get_gpt_response(system_message, user_message)

# Add GPT response to messages
messages.append({"role": "assistant", "content": gpt_response})

# Assuming chat_completion_request returns a response with tool calls
chat_response = chat_completion_request(messages, tools)

# Extract assistant's message
assistant_message = chat_response.json()["choices"][0]["message"]

# Uncomment the following lines to print the tool call content
# print("=================== Tool Call Content ===================")
# print(assistant_message["tool_calls"][0]["function"])

# Execute the tool function call
if assistant_message.get("tool_calls"):
    results = execute_function_call(assistant_message)
    messages.append({
        "role": "tool",
        "tool_call_id": assistant_message["tool_calls"][0]['id'],
        "name": assistant_message["tool_calls"][0]["function"]["name"],
        "content": results
    })

# Print the conversation
print("=================== Conversation ===================")
pretty_print_conversation(messages)

=================== Conversation ===================
system: 
<prompts>

# General Analysis Queries
1. "What are the most significant trends in viewership data over the past month? Consider views, watch time, and average view duration across different tables."
2. "Identify the top-performing videos in the last quarter based on data from the dim_top10 table."
...

# Specific Data Requests
3. "Provide a breakdown of views, watch time, and average view duration by device type for the past week, using data from raw_total and raw_gender tables."
4. "Show me the top 5 traffic sources driving the most views to the channel based on available data."
...

# SQL Query Generation
5. "Generate an SQL query to retrieve specific data points from specified tables. Clearly state the desired data points and conditions."
6. "Translate this natural language question into an SQL query: [Ask your question in plain English, specifying relevant tables]."
...

# Data Clarification
7. "Could you please clarify 

In [28]:
user_message = "What are the average view per day?"

# Messages list
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]

# Get GPT response
gpt_response = get_gpt_response(system_message, user_message)

# Add GPT response to messages
messages.append({"role": "assistant", "content": gpt_response})

# Assuming chat_completion_request returns a response with tool calls
chat_response = chat_completion_request(messages, tools)

# Extract assistant's message
assistant_message = chat_response.json()["choices"][0]["message"]

# Uncomment the following lines to print the tool call content
# print("=================== Tool Call Content ===================")
# print(assistant_message["tool_calls"][0]["function"])

# Execute the tool function call
if assistant_message.get("tool_calls"):
    results = execute_function_call(assistant_message)
    messages.append({
        "role": "tool",
        "tool_call_id": assistant_message["tool_calls"][0]['id'],
        "name": assistant_message["tool_calls"][0]["function"]["name"],
        "content": results
    })

# Print the conversation
print("=================== Conversation ===================")
pretty_print_conversation(messages)

=================== Conversation ===================
system: 
<prompts>

# General Analysis Queries
1. "What are the most significant trends in viewership data over the past month? Consider views, watch time, and average view duration across different tables."
2. "Identify the top-performing videos in the last quarter based on data from the dim_top10 table."
...

# Specific Data Requests
3. "Provide a breakdown of views, watch time, and average view duration by device type for the past week, using data from raw_total and raw_gender tables."
4. "Show me the top 5 traffic sources driving the most views to the channel based on available data."
...

# SQL Query Generation
5. "Generate an SQL query to retrieve specific data points from specified tables. Clearly state the desired data points and conditions."
6. "Translate this natural language question into an SQL query: [Ask your question in plain English, specifying relevant tables]."
...

# Data Clarification
7. "Could you please clarify 